In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from bayes_opt import BayesianOptimization

df = pd.read_csv('spaceship_titanic.csv')

df = df.drop(['Cabin'], axis=1)
df = df.drop(['Destination'], axis=1)
df = df.drop(['PassengerId'], axis=1)
df = df.drop(['Name'], axis=1)
df = df.drop(['VIP'], axis=1)
df = df.drop(['HomePlanet'], axis=1)
df = df.drop(['Age'], axis=1)

df.RoomService.fillna(df.RoomService.mean(), inplace = True)
df.FoodCourt.fillna(df.FoodCourt.mean(), inplace = True)
df.ShoppingMall.fillna(df.ShoppingMall.mean(), inplace = True)
df.Spa.fillna(df.Spa.mean(), inplace = True)
df.VRDeck.fillna(df.VRDeck.mean(), inplace = True)
df.CryoSleep.fillna(False, inplace = True)

df["Transported"].replace(True, 1, inplace=True)
df["Transported"].replace(False, 0, inplace=True)

df["CryoSleep"].replace(True, 1, inplace=True)
df["CryoSleep"].replace(False, 0, inplace=True)

for colname in df:
    df[colname] = (df[colname] - min(df[colname])) / (max(df[colname]) - min(df[colname]))

TargetFun = df["Transported"]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1772\2259509481.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.RoomService.fillna(df.RoomService.mean(), inplace = True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1772\2259509481.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [2]:
Ytrain, Ytest, Xtrain, Xtest = train_test_split(TargetFun, df.drop(["Transported"], axis=1), test_size=0.3, train_size=0.7)

## Случайный лес

In [3]:
param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier()
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='accuracy', cv=5)
grid_search_rf.fit(Xtrain, Ytrain)

print("Лучшие параметры RandomForest по решетке:", grid_search_rf.best_params_)

best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(Xtest)
accuracy_rf = accuracy_score(Ytest, y_pred_rf)
print(f"Точность RandomForest на тестовых данных (Grid Search): {accuracy_rf:.4f}\n")


Лучшие параметры RandomForest по решетке: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}
Точность RandomForest на тестовых данных (Grid Search): 0.7899



In [4]:
param_dist_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}
random_search_rf = RandomizedSearchCV(estimator=rf, param_distributions=param_dist_rf, 
                                      n_iter=10, scoring='accuracy', cv=5)
random_search_rf.fit(Xtrain, Ytrain)

print("Лучшие параметры RandomForest по случайному поиску:", random_search_rf.best_params_)

best_rf_random = random_search_rf.best_estimator_
y_pred_rf_random = best_rf_random.predict(Xtest)
accuracy_rf_random = accuracy_score(Ytest, y_pred_rf_random)
print(f"Точность RandomForest на тестовых данных (Random Search): {accuracy_rf_random:.4f}\n")


Лучшие параметры RandomForest по случайному поиску: {'n_estimators': 200, 'min_samples_split': 10, 'max_depth': 10}
Точность RandomForest на тестовых данных (Random Search): 0.7903



In [5]:
def rf_evaluate(n_estimators, max_depth, min_samples_split):
    model = RandomForestClassifier(n_estimators=int(n_estimators), max_depth=int(max_depth), 
                                   min_samples_split=int(min_samples_split))
    model.fit(Xtrain, Ytrain)
    pred = model.predict(Xtest)
    return accuracy_score(Ytest, pred)

pbounds_rf = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
}

optimizer_rf = BayesianOptimization(
    f=rf_evaluate,
    pbounds=pbounds_rf,
    random_state=42,
)

optimizer_rf.maximize(init_points=5, n_iter=15)

print("Лучшие параметры RandomForest по байесовскому подходу:", optimizer_rf.max)

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.7922    | 8.116     | 9.606     | 149.1     |
| 2         | 0.7872    | 12.37     | 3.248     | 39.64     |
| 3         | 0.74      | 2.104     | 8.929     | 124.2     |
| 4         | 0.7872    | 14.45     | 2.165     | 194.3     |
| 5         | 0.7857    | 16.82     | 3.699     | 44.55     |
| 6         | 0.7895    | 10.27     | 8.266     | 167.1     |
| 7         | 0.7926    | 9.53      | 8.761     | 149.0     |
| 8         | 0.7841    | 19.34     | 9.988     | 10.33     |
| 9         | 0.7339    | 1.002     | 8.681     | 67.6      |
| 10        | 0.7419    | 1.058     | 2.228     | 10.11     |
| 11        | 0.7343    | 1.264     | 8.8       | 186.7     |
| 12        | 0.7423    | 2.732     | 2.112     | 157.9     |
| 13        | 0.788     | 15.91     | 9.883     | 159.7     |
| 14        | 0.7887    | 19.53     | 9.619     | 172.0     |
| 15    

## XGBoost

In [6]:
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}
xgb = XGBClassifier()
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, scoring='accuracy', cv=5)
grid_search_xgb.fit(Xtrain, Ytrain)

print("Лучшие параметры XGBoost по решетке:", grid_search_xgb.best_params_)

best_xgb_grid = grid_search_xgb.best_estimator_
y_pred_xgb_grid = best_xgb_grid.predict(Xtest)
accuracy_xgb_grid = accuracy_score(Ytest, y_pred_xgb_grid)
print(f"Точность XGBoost на тестовых данных (Grid Search): {accuracy_xgb_grid:.4f}\n")

Лучшие параметры XGBoost по решетке: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 50}
Точность XGBoost на тестовых данных (Grid Search): 0.7910



In [7]:
param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}
random_search_xgb = RandomizedSearchCV(estimator=xgb, param_distributions=param_dist_xgb, 
                                       n_iter=10, scoring='accuracy', cv=5)
random_search_xgb.fit(Xtrain, Ytrain)

print("Лучшие параметры XGBoost по случайному поиску:", random_search_xgb.best_params_)

best_xgb_random = random_search_xgb.best_estimator_
y_pred_xgb_random = best_xgb_random.predict(Xtest)
accuracy_xgb_random = accuracy_score(Ytest, y_pred_xgb_random)
print(f"Точность XGBoost на тестовых данных (Random Search): {accuracy_xgb_random:.4f}\n")

Лучшие параметры XGBoost по случайному поиску: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.2}
Точность XGBoost на тестовых данных (Random Search): 0.7910



In [8]:
def xgb_evaluate(n_estimators, max_depth, learning_rate):
    model = XGBClassifier(n_estimators=int(n_estimators), max_depth=int(max_depth), learning_rate=learning_rate)
    model.fit(Xtrain, Ytrain)
    pred = model.predict(Xtest)
    return accuracy_score(Ytest, pred)

pbounds_xgb = {
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.2),
}

optimizer_xgb = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=pbounds_xgb,
    random_state=42,
)

optimizer_xgb.maximize(init_points=5, n_iter=15)

print("Лучшие параметры XGBoost по байесовскому подходу:", optimizer_xgb.max)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.7887    | 0.08116   | 9.655     | 159.8     |
| 2         | 0.791     | 0.1237    | 4.092     | 73.4      |
| 3         | 0.7899    | 0.02104   | 9.063     | 140.2     |
| 4         | 0.7891    | 0.1445    | 3.144     | 195.5     |
| 5         | 0.7926    | 0.1682    | 4.486     | 77.27     |
| 6         | 0.7891    | 0.1196    | 7.602     | 79.19     |
| 7         | 0.7903    | 0.1481    | 4.522     | 77.31     |
| 8         | 0.7937    | 0.09546   | 4.523     | 77.14     |
| 9         | 0.788     | 0.1892    | 5.575     | 83.19     |
| 10        | 0.783     | 0.1799    | 9.031     | 112.0     |
| 11        | 0.7918    | 0.1216    | 4.439     | 77.14     |
| 12        | 0.7899    | 0.03507   | 4.445     | 77.04     |
| 13        | 0.7891    | 0.1727    | 4.198     | 77.51     |
| 14        | 0.7937    | 0.1295    | 4.601     | 77.07     |
| 15    